# This NoteBook is for the GEOLOCATION to generate CSVs from RP sessions based on predefined events 

In [14]:
import os
import sys
import pprint
import csv
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import radical.entk as re
import radical.utils     as ru
import radical.analytics as ra
import radical.pilot as rp
from random import randint
import random
from matplotlib import cm
blues = cm.get_cmap(plt.get_cmap('Blues'))
greens = cm.get_cmap(plt.get_cmap('Greens'))
reds = cm.get_cmap(plt.get_cmap('Reds'))
oranges = cm.get_cmap(plt.get_cmap('Oranges'))
purples = cm.get_cmap(plt.get_cmap('Purples'))
greys = cm.get_cmap(plt.get_cmap('Greys'))


!radical-stack


/opt/anaconda2/lib/python2.7/site-packages/radical/pilot
/opt/anaconda2/lib/python2.7/site-packages/radical/utils
/opt/anaconda2/lib/python2.7/site-packages/radical/entk
/opt/anaconda2/lib/python2.7/site-packages/radical/saga
/opt/anaconda2/lib/python2.7/site-packages/radical/analytics

  python               : 2.7.14
  pythonpath           : 
  virtualenv           : /home/aymen/jupyter

  radical.analytics    : 0.70.0
  radical.entk         : 0.70.0
  radical.pilot        : 0.70.0
  radical.saga         : 0.70.0
  radical.utils        : 0.70.0



# This will not work with Design2 (only Design1)

In [ ]:
loc = "/home/aymen/SummerRadical/paper_data/Design1/entk.session-design1-54875/"
src = os.path.dirname(loc)
sid = os.path.basename(loc)
session = ra.Session(sid=sid, stype='radical.pilot', src=loc)
event_entity = 'unit'
units = session.filter(etype=event_entity, inplace=True) 
dataframe=pd.DataFrame(columns=['ImageName1','ImageName2','ImageSize1','ImageSize2','TTX','UnitID'])
dataframe2=pd.DataFrame(columns=['ImageName1','ImageName2','ImageSize1','ImageSize2','TTX','UnitID'])
durations = []
for unit in units.get():
        
        if unit.uid == 'unit.000000': #skipping the first unit because unit.00000 is only for image discovery
            skip = True
            continue
            
        else:
            
            exec_duration = unit.duration(event=[{ru.EVENT: 'exec_start'},
                                                 {ru.EVENT: 'exec_stop'}])
            
            durations.append(exec_duration)
            cu_dur = sum(durations) / len(durations)
            
            myfilepath = '/home/aymen/SummerRadical/iceberg_escience/Geolocation/Data/Geolocation_Image_pairs.csv'
                            
            with open(myfilepath) as f:
                reader = csv.reader(f) 
                next(reader) # skip header
                data = []
                for row in reader:
                    data.append(row)

                    if (unit.description["name"][11:17]) == ("RANSAC"):

                        if (row[0] == unit.description["arguments"][2]) and row[1] == unit.description["arguments"][6]:



                            try:
                                  session_data = {
                                                        'ImageName1'      : row[0],
                                                        'ImageName2'      : row[1],
                                                        'ImageSize1'      : row[2],
                                                        'ImageSize2'      : row[3],
                                                        'TTX'             : exec_duration,
                                                        'UnitID'          : unit.uid[5:]
                                                }

                            except Exception as e:
                                                print e
                                                session_data = {
                                                         'ImageName'       : row[0],
                                                         'ImageName2'      : row[1],
                                                         'ImageSize'       : row[2],
                                                         'ImageSize2'      : row[3],
                                                         'TTX'             : exec_duration,
                                                         'UnitID'          : unit.uid[5:]
                                                               }


                            dataframe =  dataframe.append(session_data, ignore_index=True)
                            dataframe.to_csv('/home/aymen/SummerRadical/iceberg_escience/Geolocation/Data/units_durations_satge1.csv',index=False)
                            
                        else :
                            pass
                        
                    else:
                        
                        if (row[0] == unit.description["arguments"][0]) and row[1] == unit.description["arguments"][5]:
                            try:
                                      session_data = {
                                                            'ImageName1'      : row[0],
                                                            'ImageName2'      : row[1],
                                                            'ImageSize1'      : row[2],
                                                            'ImageSize2'      : row[3],
                                                            'TTX'             : exec_duration,
                                                            'UnitID'          : unit.uid[5:]
                                                    }

                            except Exception as e:
                                                    print e
                                                    session_data = {
                                                             'ImageName1'      : row[0],
                                                             'ImageName2'      : row[1],
                                                             'ImageSize1'      : row[2],
                                                             'ImageSize2'      : row[3],
                                                             'TTX'             : exec_duration,
                                                             'UnitID'          : unit.uid[5:]
                                                                   }


                            dataframe2 =  dataframe2.append(session_data, ignore_index=True)
                            dataframe2.to_csv('/home/aymen/SummerRadical/iceberg_escience/Geolocation/Data/units_durations_satge0.csv',index=False)
                        else:
                            pass
    

In [2]:
loc = "/home/aymen/SummerRadical/paper_data/Design1/entk.session-design1-54875/"
src = os.path.dirname(loc)
sid = os.path.basename(loc)
rp_session = ra.Session(stype='radical.pilot', src=src)# RP session
rp_pilots = rp_session.filter(etype='pilot', inplace=False)
pilot_duration = rp_pilots.duration([rp.PMGR_ACTIVE, rp.FINAL])
rp_units = rp_session.filter(etype='unit', inplace=False)
durations = []
exec_duration = rp_units.duration(event=[{ru.EVENT: 'exec_start'}, {ru.EVENT: 'exec_stop'}])

In [ ]:
gpu_durations = []
cpu_durations = []
for unit in rp_units.get():
    
    if (unit.uid == 'unit.000000'):
        continue
        
    else:
        
        if (unit.description["name"][11:17]) == ("RANSAC"):
            
            unit.timestamps
            cpu_durations.append((unit.duration(event=[{ru.EVENT: 'exec_start'}, {ru.EVENT: 'exec_stop'}])))
        else :
            gpu_durations.append((unit.duration(event=[{ru.EVENT: 'exec_start'}, {ru.EVENT: 'exec_stop'}])/8.0)*100)
            #print unit.timestamps
            #print unit.t_start
            #print unit.t_stop
            #print unit.t_range
            #print unit.ttc

In [ ]:
unit.events

In [ ]:
owner='pilot'
consumer='unit'
resource='cores'
owner_events=[rp.AGENT_EXECUTING, rp.FINAL]
consumer_events=[{ru.EVENT: 'exec_start'}, {ru.EVENT: 'exec_stop' }]

for consumer_entity in consumers.get():

    ranges  = consumer_entity.ranges(event=consumer_events)
    cons_id = consumer_entity.uid

    consumer_nodes = consumer_entity.cfg.get('slots').get('nodes')
    resources_acquired = 0
    if resource == 'cores':
        for node in consumer_nodes:
            for cores_map in node['core_map']:
                resources_acquired += len(cores_map)
    elif resource == 'gpus':
        for node in consumer_nodes:
            for gpu_map in node['gpu_map']:
                resources_acquired += len(gpu_map)
    else:
        raise ValueError('Utilization for resource not supported')

    consumer_resources[cons_id] = resources_acquired

    # Update consumer_ranges if there is at least one range
    consumer_ranges.update({cons_id: ranges}) if len(ranges) != 0 else None

    # Sort consumer_ranges based on their values. This command
    # returns a dictionary, which is sorted based on the first value
    # of each entry. In the end the key, are out of order but the
    # values are.
    consumer_ranges = sorted(consumer_ranges.iteritems(),
                             key=lambda (k,v): (v[0],k))

    # Create a timeseries that contains all moments in consumer
    # ranges and sort. This way we have a list that has time any
    # change has happened.
    times = list()
    for cons_id,ranges in consumer_ranges:
        for r in ranges:
            times.append(r[0])
            times.append(r[1])
    times.sort()

    # we have the time sequence, now compute utilization at those points
    util = list()
    for t in times:
        cnt = 0
        for cons_id,ranges in consumer_ranges:
            for r in ranges:
                if t >= r[0] and t <= r[1]:
                    cnt += consumer_resources[cons_id]

            util.append([t, cnt])

        ret[owner_id] = {'range'      : owner_range,
                         'resources'  : owner_resources,
                         'utilization': util}

In [3]:
util_cpu = rp_session.utilization(owner='pilot', consumer='unit', resource='cores',owner_events=[rp.AGENT_EXECUTING, rp.FINAL], consumer_events=[{ru.EVENT: 'exec_start'}, {ru.EVENT: 'exec_stop' }])

{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu020'}
{u'uid': u'gpu026', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu026'}
{u'uid': u'gpu026', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu026'}
{u'uid': u'gpu023', u'gpu_map': 

{u'uid': u'gpu021', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu021'}
{u'uid': u'gpu026', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu026'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': 

{u'uid': u'gpu023', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu023'}
{u'uid': u'gpu021', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu021'}
{u'uid': u'gpu021', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu021'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[2]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[2]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[2]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[2]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': 

{u'uid': u'gpu023', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu023'}
{u'uid': u'gpu026', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu026'}
{u'uid': u'gpu023', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu023'}
{u'uid': u'gpu020', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_

{u'uid': u'gpu021', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu021'}
{u'uid': u'gpu023', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu023'}
{u'uid': u'gpu023', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu023'}
{u'uid': u'gpu023', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu023'}
{u'uid': u'gpu020', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu021', 

...

{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu021', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu021'}
{u'uid': u'gpu026', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu026'}
{u'uid': u'gpu020', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu020'}
{u'uid': u'gpu023', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu023'}
{u'uid': u'gpu020', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu021', u'g

{u'uid': u'gpu023', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu023'}
{u'uid': u'gpu026', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu026'}
{u'uid': u'gpu023', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu023'}
{u'uid': u'gpu021', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu021'}
{u'uid': u'gpu020', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu026', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu026'}
{u'uid': u'gpu020', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu021', 

{u'uid': u'gpu026', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu026'}
{u'uid': u'gpu023', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu023'}
{u'uid': u'gpu021', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu021'}
{u'uid': u'gpu026', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu026'}
{u'uid': u'gpu021', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu021'}
{u'uid': u'gpu026', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu026'}
{u'uid': u'gpu021', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu021'}
{u'uid': u'gpu021', 

{u'uid': u'gpu021', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu021'}
{u'uid': u'gpu021', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu021'}
{u'uid': u'gpu023', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu023'}
{u'uid': u'gpu026', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu026'}
{u'uid': u'gpu026', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu026'}
{u'uid': u'gpu026', u'gpu_map': [[1]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[1]], u'name': u'gpu026'}
{u'uid': u'gpu023', u'gpu_map': [[0]], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu023'}
{u'uid': u'gpu020', 

{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [], u'mem': 0, u'lfs': {u'path': u'/local/7636582', u'size': 0}, u'core_map': [[0]], u'name': u'gpu020'}
{u'uid': u'gpu020', u'gpu_map': [[1]], u'

In [4]:
Des1cpu = np.array(util_cpu['pilot.0000']['utilization'])
np.save('/home/aymen/Des1CpuUtil',Des1cpu)

In [7]:
util_gpu = rp_session.utilization(owner='pilot', consumer='unit', resource='gpus',owner_events=[rp.AGENT_EXECUTING, rp.FINAL], consumer_events=[{ru.EVENT: 'exec_start'}, {ru.EVENT: 'exec_stop' }])

In [8]:
Des1gpu = np.array(util_gpu['pilot.0000']['utilization'])
np.save('/home/aymen/Des1GpuUtil',Des1gpu)

In [9]:
Design1CpuUtil= np.load('/home/aymen/Des1CpuUtil.npy')
Design1GpuUtil = np.load('/home/aymen/Des1GpuUtil.npy')

In [10]:
stime_des1_cpu = Design1CpuUtil[0][0] 
print(stime_des1_cpu)
for i in range(len(Design1CpuUtil)):
    tt_c = Design1CpuUtil[i][0] - stime_des1_cpu
    Design1CpuUtil[i][0] = tt_c

112.32961297035217


In [11]:
stime_des1_gpu = Design1GpuUtil[0][0] 
print(stime_des1_gpu)
for i in range(len(Design1GpuUtil)):
    tt_g = Design1GpuUtil[i][0] - stime_des1_gpu
    Design1GpuUtil[i][0] = tt_g

112.32961297035217
